In [1]:
import gurobipy as gp
from itertools import product
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Test gurobipy

In [2]:
# Create a new model
m = gp.Model()

# Create variables
x = m.addVar(vtype='B', name="x")
y = m.addVar(vtype='B', name="y")
z = m.addVar(vtype='B', name="z")

# Set objective function
m.setObjective(x + y + 2 * z, gp.GRB.MAXIMIZE)

# Add constraints
m.addConstr(x + 2 * y + 3 * z <= 4)
m.addConstr(x + y >= 1)

# Solve it!
m.optimize()

print(f"Optimal objective value: {m.objVal}")
print(f"Solution values: x={x.X}, y={y.X}, z={z.X}")

Set parameter TokenServer to value "dev.cma.mines-paristech.fr"
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
Optimal objective value: 3.0
Solution values: x=1.0, y=0.0, z=1.0


## 1 Modèle de base des centrales thermiques

### 1.1 Planification journalière du parc thermique

In [3]:
class Centrale:
    def __init__(self, name, N, Pmin, Pmax, Cmwh):
        self.name = name
        self.N = N
        self.Pmin = Pmin
        self.Pmax = Pmax
        self.Cmwh = Cmwh

In [4]:
dict_11 = {}
dict_11["A"] = Centrale(
    name="A",
    N=12,
    Pmax=2000,
    Pmin=850,
    Cmwh=1.5
)
dict_11["B"] = Centrale(
    name="B",
    N=10,
    Pmax=1750,
    Pmin=1250,
    Cmwh=1.38
)
dict_11["C"] = Centrale(
    name="C",
    N=5,
    Pmax=4000,
    Pmin=1500,
    Cmwh=2.75
)

In [5]:
model = gp.Model(name="1.1")

In [6]:
from unicodedata import name


dict_N = {}
dict_P = {}

for t in range(24):
    for X in dict_11:
        dict_N[X,t] = model.addVar(lb=0,ub=dict_11[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} allumées à {t}h")
    for X in dict_11:
        dict_P[X,t] = model.addVar(name=f"Puissance totale {X} à {t}h")


In [7]:
for element in product(dict_11,range(24)):
    dict_P[element[0],element[1]]

In [8]:

model.setObjective(gp.quicksum([dict_P[element[0],element[1]] * dict_11[element[0]].Cmwh for element in product(dict_11,range(24))]), gp.GRB.MINIMIZE)

In [9]:
for t in range(24):
    for X in dict_11:
        model.addConstr(dict_P[X,t]<=dict_N[X,t] * dict_11[X].Pmax, name=f"borne sup puissance, {X} à {t}h")
        model.addConstr(dict_P[X,t]>=dict_N[X,t] * dict_11[X].Pmin, name=f"borne inf puissance, {X} à {t}h")
    if t < 6:
        model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==15000)
    if (t >= 6)&(t<9):
        model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==30000)
    if (t >= 9)&(t<15):
        model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==25000)
    if (t >= 15)&(t<18):
        model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==40000)
    if t >= 18:
        model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==27000)

In [10]:
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 144 columns and 360 nonzeros
Model fingerprint: 0xe0a5527a
Variable types: 72 continuous, 72 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 3e+00]
  Bounds range     [5e+00, 1e+01]
  RHS range        [2e+04, 4e+04]
Found heuristic solution: objective 1235375.0000
Presolve removed 162 rows and 139 columns
Presolve time: 0.01s
Presolved: 6 rows, 5 columns, 14 nonzeros
Found heuristic solution: objective 881275.00000
Variable types: 2 continuous, 3 integer (0 binary)

Root relaxation: objective 8.694000e+05, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 869400.000    0    1 881275.000 869400.000  1.35%     -    

In [11]:
print(f"Coût : {model.ObjVal}")

Coût : 869400.0


In [12]:
df = pd.DataFrame()
df["h"] = range(24) 
df["Consommation (MW)"] = np.array([15,15,15,15,15,15,30,30,30,25,25,25,25,25,25,40,40,40,27,27,27,27,27,27])*1000
df["Production total"] = 0
df["Coût total"] = 0
for X in dict_11:
    df[f"Nb centrale {X}"] = [int(dict_N[X,t].X) for t in range(24)]
    df[f"Puissance tot {X}"] = [dict_P[X,t].X for t in range(24)]
    df[f"Coût {X}"] = [dict_P[X,t].X * dict_11[X].Cmwh for t in range(24)]
    df["Production total"] += df[f"Puissance tot {X}"]
    df["Coût total"] += df[f"Coût {X}"]
df["Coût MWh"] = df["Coût total"]/df["Production total"]

In [55]:
from turtle import color
import plotly.graph_objects as go

fig = go.Figure()

Y = df["Consommation (MW)"].copy()*0
for X in dict_11:
    Y += df[f"Puissance tot {X}"]
    fig.add_trace(
        go.Scatter(
        x=df["h"], 
        y=Y, 
        fill='tonexty',
        name = f"Production {X}"
        )
    )

# fig.add_trace(
#     go.Scatter(
#         x = df["h"],
#         y = df["Consommation (MW)"],
#         name = "Consommation"
#     )
# )
fig.add_trace(
    go.Scatter(
        x = df["h"],
        y = df["Production total"],
        name="Production total",
        line = dict(dash='dash',color="red")
    )
)
fig.add_trace(
    go.Scatter(
        x=df["h"],
        y=df["Coût MWh"],
        yaxis="y2",
        name="Coût MWh",
        line_color="black"
    )
)

fig.update_layout(
    hovermode='x',
    yaxis=dict(title="MW" ,range=[0,df["Consommation (MW)"].max()*1.1]),
    yaxis2=dict(title="€/MWh",
    range=[df["Coût MWh"].min()*0.95,df["Coût MWh"].max()*1.1],
    anchor="free",
    overlaying="y",
    side="right",
    position=1
    ),
    title = "Répartition de la production électrique dans la journée"
)
fig
fig.show()